In [ ]:
import pyspark
from pyspark.sql import SparkSession
import os

# this apparently works and does not require the sql extensions
spark = SparkSession.builder \
    .master("local") \
    .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.0') \
    .config("spark.sql.catalog.spark_catalog","org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "./warehouse") \
    .config("spark.sql.defaultCatalog", "local") \
    .getOrCreate()


In [ ]:
# create a table
sql = """
CREATE or replace TABLE local.b1.employees (first_name string, last_name string, age int) using iceberg
"""
spark.sql(sql).show()

In [ ]:
sql = """
INSERT INTO local.b1.employees values ('Tom','Brady',42)
"""
spark.sql(sql)

In [ ]:
spark.sql("update local.b1.employees set age = 56")

In [ ]:
spark.sql("delete from local.b1.employees where age = 4")

In [ ]:
spark.sql("select * from local.b1.employees").show()

In [ ]:
spark.sql("create or replace table local.b1.employees2 as select * from local.b1.employees")

In [ ]:
sql = """
    merge into local.b1.employees as tgt
        using local.b1.employees2 as src
            on tgt.first_name = src.first_name
    when not matched then insert *
"""
spark.sql(sql)